## Functions

In [1]:
# Converts a number in base10 to base2.

def to_binary(num_10):
  N = num_10
  output = ''

  while N > 1:
    if N % 2 == 1:
      output += '1'
      N = (N-1) // 2
      continue

    N = N // 2
    output += '0'
  output += '1'

  return output[::-1]

# Euclid's algorithm.
def gcd(a, b):
  if a < b:
    a, b = b, a
  # Get the remainder when a is divided by b: a = b*k + r, where 0 <= r < b.
  r = a % b
  # If r = 0 then b | a and we are done.
  if r == 0:
    return b
  # If b does not divide a then the gcd(a, b) will be the largest integer dividing b and r.
  return gcd(b, r)

# (!) DO NOT USE. The Bezout coefficients are wrong.
# Need to figure out why this is wrong.
def wrong_ext_gcd(a, b):
  if a < b:
    a, b = b, a

  # Initialize.
  s_prev, t_prev = 1, 0
  s, t = 0, 1
  q = a // b
  r_prev, r = b, a % b

  while True:
    # We will need to update both s, t and s_next, t_next
    s_tmp, t_tmp = s, t
    # Update step is s_n = s_{n-2} - q_n*s_{n-1} and similarly for t_n.
    s, t = s_prev - q*s, t_prev - q*t
    # Update s, t
    s_prev, t_prev = s_tmp, t_tmp
    print(s_prev, t_prev)
    if r == 0:
      return r_prev, s, t
    # Else update r's and q.
    # r != 0
    q = r_prev // r
    r_prev, r = r, r_prev % r

# Correct implementation. This works for the same reason that the gcd algorithm works:
# because gcd(a, b) = gcd(b, a mod b). Assuming that gcd(b, a mod b) = bx + (a mod b)y,
# substitute a mod b = a - a // b to get that gcd(a, b) = ay + b(x - (a // b)*y).
def ext_gcd(a, b):
  if b == 0:
    return (a, 1, 0)
  gcd, x1, y1 = ext_gcd(b, a % b)
  x = y1
  y = x1 - (a // b) * y1
  return (gcd, x, y)

def legendre_symbol(a, p):
    """Compute the Legendre symbol (a | p)"""
    ls = pow(a, (p - 1) // 2, p)
    return -1 if ls == p - 1 else ls

# Tonelli-Shanks algorithm (ChatGPT)
def tonelli_shanks(a, p):
    """
    Solve for x such that x^2 ≡ a mod p, using the Tonelli–Shanks algorithm.
    Requires that p is an odd prime and a is a quadratic residue mod p.
    Returns one solution (the other is p - x).
    Raises ValueError if no square root exists.
    """
    if legendre_symbol(a, p) != 1:
        raise ValueError(f"{a} is not a square mod {p}")

    # Handle the easy case
    if p % 4 == 3:
        return pow(a, (p + 1) // 4, p)

    # Step 1: Factor p-1 = Q * 2^S with Q odd
    Q = p - 1
    S = 0
    while Q % 2 == 0:
        Q //= 2
        S += 1

    # Step 2: Find a non-residue z
    for z in range(2, p):
        if legendre_symbol(z, p) == -1:
            break
    c = pow(z, Q, p)

    # Step 3: Initialize variables
    R = pow(a, (Q + 1) // 2, p)
    t = pow(a, Q, p)
    M = S

    # Step 4: Loop until t == 1
    while t != 1:
        # Find the least i such that t^(2^i) ≡ 1
        i = 1
        temp = pow(t, 2, p)
        while temp != 1:
            temp = pow(temp, 2, p)
            i += 1
            if i == M:
                raise Exception("No solution found")

        # Update variables
        b = pow(c, 1 << (M - i - 1), p)
        R = (R * b) % p
        t = (t * b * b) % p
        c = (b * b) % p
        M = i

    return R


## Modular Exponentiation

In [2]:
pow(101, 17, 22663)

19906

## Public Keys
The most common value for e is 65537 = 0x10001.

In [3]:
p, q = 17, 23
N = p*q
e= 65537
# Encrypt 12.
pow(12, e, N)

301

## Euler's Totient
If p or q is deduced, then $\phi(N) = (p-1)(q-1)$ can be deduced which allows for messages to be decrypted.

In [4]:
p = 857504083339712752489993810777
q = 1029224947942998075080348647219

In [6]:
flag = (p-1) * (q-1)
flag

882564595536224140639625987657529300394956519977044270821168

## Private Keys

Note that in RSA, the private key is $d = e^{-1}\mod\phi(N)$.

In [10]:
p = 857504083339712752489993810777
q = 1029224947942998075080348647219
e=65537
totient = (p-1) * (q-1)

In [14]:
d = pow(e, -1, totient)

In [15]:
d

121832886702415731577073962957377780195510499965398469843281

## RSA Decryption
We'll use the private key $d$ computed in the previous challenge to decrypt c.Note that $c \equiv M^e\mod N$ So $M \equiv c^d\mod N$.

In [13]:
N = 882564595536224140639625987659416029426239230804614613279163
e = 65537
c = 77578995801157823671636298847186723593814843845525223303932

In [16]:
pow(c, d, N)

13371337

## RSA Signatures

First encrypt via $c = m^{e_0}\mod N_0$ where $e_0$ is the receiver's public key. Then to sign a message, you compute a hash $S = H(m)^{d_1}$ where $d_1$ is your private key. Your friend: decrypts the message using their private key, and verifies the sender using your public key.

We need to sign crypto{Immut4ble_m3ssag1ng} using SHA256.

In [17]:
N = 15216583654836731327639981224133918855895948374072384050848479908982286890731769486609085918857664046075375253168955058743185664390273058074450390236774324903305663479046566232967297765731625328029814055635316002591227570271271445226094919864475407884459980489638001092788574811554149774028950310695112688723853763743238753349782508121985338746755237819373178699343135091783992299561827389745132880022259873387524273298850340648779897909381979714026837172003953221052431217940632552930880000919436507245150726543040714721553361063311954285289857582079880295199632757829525723874753306371990452491305564061051059885803
d = 11175901210643014262548222473449533091378848269490518850474399681690547281665059317155831692300453197335735728459259392366823302405685389586883670043744683993709123180805154631088513521456979317628012721881537154107239389466063136007337120599915456659758559300673444689263854921332185562706707573660658164991098457874495054854491474065039621922972671588299315846306069845169959451250821044417886630346229021305410340100401530146135418806544340908355106582089082980533651095594192031411679866134256418292249592135441145384466261279428795408721990564658703903787956958168449841491667690491585550160457893350536334242689

In [26]:
!pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.2 MB/s eta 0:00:00


In [27]:
from Crypto.Util.Padding import pad, unpad
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

In [29]:
from Crypto.Util.number import bytes_to_long
# test
bytes_to_long(b'11111')

211278704945

In [34]:
flag = b'crypto{Immut4ble_m3ssag1ng}'
M_int = bytes_to_long(flag)

In [33]:
pow(M_int, d, N)

7291586416201791766037636146241032964444936655920786498190334983618088824671059306356976964930175153467659405016015484148997139426801809505209218745817283306679724121955906940007795458080780539944591259177980185221220905794340880634553922224851374662498660988101250385874724529886300501437862749328840700899496861012259366168737844891835419680934283978224510379976627338846422811241945389667155869113639301247183460690434224070987806621763555532207354013586001527640073178859649682724810390492935602724187386492928658632710195517150986345438635341713740498090921365843196435262342784637782024652806738889588590356660

In [36]:
# We need to compute H(m) using SHA256 hash.
from Crypto.Hash import SHA256

hash_object = SHA256.new(data=flag)
print(hash_object.digest())
print(hash_object.hexdigest())

b'\x99\xb4\xc7\xbb\x81L\xc60\xc4\x19\x9eH\x14\xff\xed\x85\xa85\xf6O\xfc\x82\xaa\xda\xa68\x8d\x9d\xf9\xae\xb2\xcb'
99b4c7bb814cc630c4199e4814ffed85a835f64ffc82aadaa6388d9df9aeb2cb


In [37]:
# Now we compute H(m)^d mod N.
Hm = b'\x99\xb4\xc7\xbb\x81L\xc60\xc4\x19\x9eH\x14\xff\xed\x85\xa85\xf6O\xfc\x82\xaa\xda\xa68\x8d\x9d\xf9\xae\xb2\xcb'
pow(bytes_to_long(Hm), d, N)

13480738404590090803339831649238454376183189744970683129909766078877706583282422686710545217275797376709672358894231550335007974983458408620258478729775647818876610072903021235573923300070103666940534047644900475773318682585772698155617451477448441198150710420818995347235921111812068656782998168064960965451719491072569057636701190429760047193261886092862024118487826452766513533860734724124228305158914225250488399673645732882077575252662461860972889771112594906884441454355959482925283992539925713424132009768721389828848907099772040836383856524605008942907083490383109757406940540866978237471686296661685839083475

## Factoring

In [71]:
!pip install primefac

In [39]:
N = 510143758735509025530880200653196460532653147

In [44]:
from primefac import primegen

#test
list(primegen(limit=20))

[2, 3, 5, 7, 11, 13, 17, 19]

In [62]:
# The hint is suggesting that we use a fork of primefac.
!pip install factordb-python

  Preparing metadata (setup.py) ... done
  Created wheel for factordb-python: filename=factordb_python-1.3.0-py3-none-any.whl size=4620 sha256=3ca0acb9759e631f288c792600370c9ee3a80da5a188ed792618379a354426fb
  Stored in directory: /root/.cache/pip/wheels/23/c6/8b/5b4f1a12b84694034056342ea171aabf5c985f6438bcc7cceb
Successfully built factordb-python


In [67]:
# Almost instant!

from factordb.factordb import FactorDB

f = FactorDB(N)

print(f.get_factor_list())

print(f.connect())

print(f.get_factor_list())

print(f.get_factor_from_api())

f.get_status()


[]
<Response [200]>
[19704762736204164635843, 25889363174021185185929]
[['19704762736204164635843', 1], ['25889363174021185185929', 1]]


'FF'

In [68]:
19704762736204164635843 < 25889363174021185185929

True

## Monoprime

In [69]:
n = 171731371218065444125482536302245915415603318380280392385291836472299752747934607246477508507827284075763910264995326010251268493630501989810855418416643352631102434317900028697993224868629935657273062472544675693365930943308086634291936846505861203914449338007760990051788980485462592823446469606824421932591
e = 65537
ct = 161367550346730604451454756189028938964941280347662098798775466019463375610700074840105776873791605070092554650190486030367121011578171525759600774739890458414593857709994072516290998135846956596662071379067305011746842247628316996977338024343628757374524136260758515864509435302781735938531030576289086798942


In [74]:
from primefac import isprime

isprime(n)

True

n is a prime here, rather than a product of two primes. That means finding the modular inverse is a lot easer since $\phi(n) = n-1$.

In [75]:
d = pow(e, -1, n-1)
pow(ct, d, n)

44981230718212183184022261350591509650967020174777710365581497711727767219325

In [78]:
from Crypto.Util.number import long_to_bytes
long_to_bytes(44981230718212183184022261350591509650967020174777710365581497711727767219325)

b'crypto{0n3_pr1m3_41n7_pr1m3_l0l}'

## Manyprime

Here the modulus will be a product of 30 primes. But apparently, it is susceptible to Elliptic Curve factorization. Here we use Sage built in EC factorization method.

In [80]:
# I don't think there is sage support for notebooks, or even Windows. Have to do
# this in WSL: see https://doc.sagemath.org/html/en/installation/index.html.

In [81]:
n = 580642391898843192929563856870897799650883152718761762932292482252152591279871421569162037190419036435041797739880389529593674485555792234900969402019055601781662044515999210032698275981631376651117318677368742867687180140048715627160641771118040372573575479330830092989800730105573700557717146251860588802509310534792310748898504394966263819959963273509119791037525504422606634640173277598774814099540555569257179715908642917355365791447508751401889724095964924513196281345665480688029639999472649549163147599540142367575413885729653166517595719991872223011969856259344396899748662101941230745601719730556631637
e = 65537
ct = 320721490534624434149993723527322977960556510750628354856260732098109692581338409999983376131354918370047625150454728718467998870322344980985635149656977787964380651868131740312053755501594999166365821315043312308622388016666802478485476059625888033017198083472976011719998333985531756978678758897472845358167730221506573817798467100023754709109274265835201757369829744113233607359526441007577850111228850004361838028842815813724076511058179239339760639518034583306154826603816927757236549096339501503316601078891287408682099750164720032975016814187899399273719181407940397071512493967454225665490162619270814464

In [82]:
# Sage actually did it in 43 seconds:
factors = [9282105380008121879,
 9303850685953812323,
 9389357739583927789,
 10336650220878499841,
 10638241655447339831,
 11282698189561966721,
 11328768673634243077,
 11403460639036243901,
 11473665579512371723,
 11492065299277279799,
 11530534813954192171,
 11665347949879312361,
 12132158321859677597,
 12834461276877415051,
 12955403765595949597,
 12973972336777979701,
 13099895578757581201,
 13572286589428162097,
 14100640260554622013,
 14178869592193599187,
 14278240802299816541,
 14523070016044624039,
 14963354250199553339,
 15364597561881860737,
 15669758663523555763,
 15824122791679574573,
 15998365463074268941,
 16656402470578844539,
 16898740504023346457,
 17138336856793050757,
 17174065872156629921,
 17281246625998849649]

In [86]:
# Check that the algorithm gave the right factors:
num = 1

for p in factors:
  num *= p
num == n

True

If $n = p_1\cdots p_n$ then $\phi(n) = (p_1-1)\cdots (p_n-1)$.

In [87]:
totient = 1
for p in factors:
  totient *= p-1

d = pow(e, -1, totient)
d

550678668057320270486894977713280661612386679473387004433937990192257019009907567239119014475553352525151197453709254404700764265858700310364694278627740936496980796299768511049032643264376359955800264702060642943865098798169536307529141678474309717202937979408478029523743103567031688359785501822834619529958535700930643075021264741270961281670213783654512824458687819377344959486643077970750359994201843167516881424941358993141613678672658457307473999414959060492054590697942850267611629510044892244182678685166778803001599034675379574122977541489674412812063097749209824025650218295674398899691008132810473473

In [90]:
flag = pow(ct, d, n)
long_to_bytes(flag) # those are small factors?...

b'crypto{700_m4ny_5m4ll_f4c70r5}'

## Salty

In [98]:
#!/usr/bin/env python3

from Crypto.Util.number import getPrime, inverse, bytes_to_long, long_to_bytes

e = 1
d = -1

while d == -1:
    p = getPrime(512)
    q = getPrime(512)
    phi = (p - 1) * (q - 1)
    d = inverse(e, phi)

n = p * q

flag = b"XXXXXXXXXXXXXXXXXXXXXXX"
pt = bytes_to_long(flag)
ct = pow(pt, e, n)

print(f"n = {n}")
print(f"e = {e}")
print(f"ct = {ct}")

pt = pow(ct, d, n)
decrypted = long_to_bytes(pt)
assert decrypted == flag


n = 98868319883029907016349725196897151600662217009690343400705449197683613662916936195675900019840288832356302268775528808231764194264376008919687675765031948836733924247422191003599619683904135959427565804452993259710635451580260389715246246232044973466736769434300739350920996294285435514326836521889377314113
e = 1
ct = 8461779300153613774778637702853471884451704643272595544


In [99]:
#output.txt file
n = 110581795715958566206600392161360212579669637391437097703685154237017351570464767725324182051199901920318211290404777259728923614917211291562555864753005179326101890427669819834642007924406862482343614488768256951616086287044725034412802176312273081322195866046098595306261781788276570920467840172004530873767
e = 1
ct = 44981230718212183604274785925793145442655465025264554046028251311164494127485


In [100]:
long_to_bytes(ct)

b'crypto{saltstack_fell_for_this!}'

## Modulus Inutilis

In [109]:
#!/usr/bin/env python3

# from Crypto.Util.number import getPrime, inverse, bytes_to_long, long_to_bytes

# e = 3
# d = -1

# while d == -1:
#     p = getPrime(1024)
#     q = getPrime(1024)
#     phi = (p - 1) * (q - 1)
#     d = inverse(e, phi)

# n = p * q

# flag = b"XXXXXXXXXXXXXXXXXXXXXXX"
# pt = bytes_to_long(flag)
# ct = pow(pt, e, n)

# print(f"n = {n}")
# print(f"e = {e}")
# print(f"ct = {ct}")

# pt = pow(ct, d, n)
# decrypted = long_to_bytes(pt)
# assert decrypted == flag


In [102]:
n = 17258212916191948536348548470938004244269544560039009244721959293554822498047075403658429865201816363311805874117705688359853941515579440852166618074161313773416434156467811969628473425365608002907061241714688204565170146117869742910273064909154666642642308154422770994836108669814632309362483307560217924183202838588431342622551598499747369771295105890359290073146330677383341121242366368309126850094371525078749496850520075015636716490087482193603562501577348571256210991732071282478547626856068209192987351212490642903450263288650415552403935705444809043563866466823492258216747445926536608548665086042098252335883
e = 3
ct = 243251053617903760309941844835411292373350655973075480264001352919865180151222189820473358411037759381328642957324889519192337152355302808400638052620580409813222660643570085177957


In [104]:
# The ciphertext is cleary much smaller than n. Just take the normal cube root.
from sympy import root

root(ct, 3)

624239975241694158443315202759206900318542905782320965248893

In [106]:
flag = long_to_bytes(624239975241694158443315202759206900318542905782320965248893)
flag

b'crypto{N33d_m04R_p4dd1ng}'

## Working with fields

In [107]:
pow(209, -1, 991)

569

## Generators of groups

Find the generator of $F_p^\times$ given that p = 28151.